In [34]:
import pandas as pd
import os
from scipy.stats import chi2_contingency
import numpy as np
from meteostat import Hourly, Point
from datetime import datetime
from astral import LocationInfo
from astral.sun import sun
from datetime import date

### temperatura i zachmurzenie

In [35]:
parh_to_temp = os.getcwd() + '\\dane_temp'
files = [os.path.join(parh_to_temp, file) for file in os.listdir(parh_to_temp) if file.endswith('.csv')]

column_names = ["Kod stacji", "Nazwa stacji", "Rok", "Miesiąc", "Dzień", 
                "Maksymalna temperatura dobowa [°C]", "Status pomiaru TMAX",
                "Minimalna temperatura dobowa [°C]", "Status pomiaru TMIN",
                "Średnia temperatura dobowa [°C]", "Status pomiaru STD",
                "Temperatura minimalna przy gruncie [°C]", "Status pomiaru TMNG",
                "Suma dobowa opadu [mm]", "Status pomiaru SMDB",
                "Rodzaj opadu [S/W/ ]", "Wysokość pokrywy śnieżnej [cm]",
                "Status pomiaru PKSN", "Równoważnik wodny śniegu  [mm/cm]",
                "Status pomiaru RWSN", "Usłonecznienie [godziny]",
                "Status pomiaru USL", "Czas trwania opadu deszczu [godziny]",
                "Status pomiaru DESZ", "Czas trwania opadu śniegu [godziny]",
                "Status pomiaru SNEG", "Czas trwania opadu deszczu ze śniegiem [godziny]",
                "Status pomiaru DISN", "Czas trwania gradu [godziny]",
                "Status pomiaru GRAD", "Czas trwania mgły [godziny]",
                "Status pomiaru MGLA", "Czas trwania zamglenia  [godziny]",
                "Status pomiaru ZMGL", "Czas trwania sadzi [godziny]",
                "Status pomiaru SADZ", "Czas trwania gołoledzi [godziny]",
                "Status pomiaru GOLO", "Czas trwania zamieci śnieżnej niskiej [godziny]",
                "Status pomiaru ZMNI", "Czas trwania zamieci śnieżnej wysokiej [godziny]",
                "Status pomiaru ZMWS", "Czas trwania zmętnienia [godziny]",
                "Status pomiaru ZMET", "Czas trwania wiatru >=10m/s [godziny]",
                "Status pomiaru FF10", "Czas trwania wiatru >15m/s [godziny]",
                "Status pomiaru FF15", "Czas trwania burzy  [godziny]",
                "Status pomiaru BRZA", "Czas trwania rosy  [godziny]",
                "Status pomiaru ROSA", "Czas trwania szronu [godziny]",
                "Status pomiaru SZRO", "Wystąpienie pokrywy śnieżnej  [0/1]",
                "Status pomiaru DZPS", "Wystąpienie błyskawicy  [0/1]",
                "Status pomiaru DZBL", "Stan gruntu [Z/R]",
                "Izoterma dolna  [cm]", "Status pomiaru IZD",
                "Izoterma górna [cm]", "Status pomiaru IZG",
                "Aktynometria  [J/cm2]", "Status pomiaru AKTN"]

In [36]:
from IPython.display import display
import pandas as pd

df_combined_1 = pd.DataFrame()
for file in files:
    print(file)
    data = pd.read_csv(file, encoding='latin1')
    data.columns = column_names
    data = data[column_names]
    df_combined_1 = pd.concat([df_combined_1, data], axis=0)

df_combined_1 = df_combined_1[df_combined_1["Nazwa stacji"].isin(["KO£OBRZEG-D\x8fWIRZYNO","KO£OBRZEG-D\x8fWIRZYNO           "])]

# Create datetime column
df_combined_1['Data'] = pd.to_datetime(df_combined_1[['Rok', 'Miesiąc', 'Dzień']].astype(str).agg('-'.join, axis=1))

# Create a new dataframe with hourly data
df_hourly = pd.DataFrame()
for idx, row in df_combined_1.iterrows():
    for hour in range(24):
        new_row = row.copy()
        new_row['Godzina'] = hour
        df_hourly = pd.concat([df_hourly, pd.DataFrame([new_row])], ignore_index=True)

# Cleaning data
columns_to_drop = ['Rok', 'Miesiąc', 'Dzień', 'Kod stacji', 'Nazwa stacji']
status_columns = [col for col in df_hourly.columns if 'Status pomiaru' in col]
columns_to_drop.extend(status_columns)
df_combined_1 = df_hourly.drop(columns=columns_to_drop)

df_combined_1 = df_combined_1.sort_values(['Data', 'Godzina'])
display(df_combined_1)

c:\Users\micha\Desktop\Magisterka\dane_temp\s_d_01_2025.csv
c:\Users\micha\Desktop\Magisterka\dane_temp\s_d_02_2025.csv
c:\Users\micha\Desktop\Magisterka\dane_temp\s_d_100_2023.csv
c:\Users\micha\Desktop\Magisterka\dane_temp\s_d_100_2024.csv


,Maksymalna temperatura dobowa [°C],Minimalna temperatura dobowa [°C],Średnia temperatura dobowa [°C],Temperatura minimalna przy gruncie [°C],Suma dobowa opadu [mm],Rodzaj opadu [S/W/ ],Wysokość pokrywy śnieżnej [cm],Równoważnik wodny śniegu [mm/cm],Usłonecznienie [godziny],Czas trwania opadu deszczu [godziny],...,Czas trwania rosy [godziny],Czas trwania szronu [godziny],Wystąpienie pokrywy śnieżnej [0/1],Wystąpienie błyskawicy [0/1],Stan gruntu [Z/R],Izoterma dolna [cm],Izoterma górna [cm],Aktynometria [J/cm2],Data,Godzina
1416,13.7,6.7,9.6,9.4,0.0,NaN,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,R,0.0,0.0,0.0,2023-01-02,0
1417,13.7,6.7,9.6,9.4,0.0,NaN,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,R,0.0,0.0,0.0,2023-01-02,1
1418,13.7,6.7,9.6,9.4,0.0,NaN,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,R,0.0,0.0,0.0,2023-01-02,2
1419,13.7,6.7,9.6,9.4,0.0,NaN,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,R,0.0,0.0,0.0,2023-01-02,3
1420,13.7,6.7,9.6,9.4,0.0,NaN,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,R,0.0,0.0,0.0,2023-01-02,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1411,4.8,3.2,3.6,2.1,0.1,W,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,R,NaN,NaN,NaN,2025-02-28,19
1412,4.8,3.2,3.6,2.1,0.1,W,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,R,NaN,NaN,NaN,2025-02-28,20
1413,4.8,3.2,3.6,2.1,0.1,W,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,R,NaN,NaN,NaN,2025-02-28,21
1414,4.8,3.2,3.6,2.1,0.1,W,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,R,NaN,NaN,NaN,2025-02-28,22


### temperatura z biblioteki python

In [37]:
# Create Point for Warsaw
warsaw = Point(52.2297, 21.0122)

# Get date range from df_combined_1
start = df_combined_1['Data'].min()
end = df_combined_1['Data'].max()

# Get hourly data
data = Hourly(warsaw, start, end)
df_temperature = data.fetch()

# Reset index to make the datetime a column
df_temperature = df_temperature.reset_index()

# Rename columns for clarity
df_temperature = df_temperature.rename(columns={'time': 'Data', 'temp': 'Temperatura'})
df_temperature["Godzina"] = df_temperature["Data"].dt.hour
# Display first few rows    
display(df_temperature)

,Data,Temperatura,dwpt,rhum,prcp,snow,wdir,wspd,wpgt,pres,tsun,coco,Godzina
0,2023-01-02 00:00:00,9.5,5.9,78.0,0.0,NaN,180.0,3.6,13.0,1021.9,NaN,4.0,0
1,2023-01-02 01:00:00,9.0,5.7,80.0,0.0,NaN,160.0,3.6,13.0,1022.0,NaN,3.0,1
2,2023-01-02 02:00:00,9.5,5.9,78.0,0.0,NaN,150.0,3.6,11.1,1021.4,NaN,3.0,2
3,2023-01-02 03:00:00,7.7,5.8,88.0,0.0,NaN,130.0,3.6,11.1,1021.4,NaN,3.0,3
4,2023-01-02 04:00:00,7.6,5.7,88.0,0.0,NaN,130.0,3.6,11.1,1021.5,NaN,4.0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
18908,2025-02-27 20:00:00,7.3,3.9,79.0,0.0,NaN,340.0,3.6,9.3,1018.4,NaN,4.0,20
18909,2025-02-27 21:00:00,6.6,4.1,84.0,0.0,NaN,270.0,3.6,9.3,1018.5,NaN,4.0,21
18910,2025-02-27 22:00:00,5.9,4.1,88.0,0.0,NaN,280.0,3.6,9.3,1018.5,NaN,4.0,22
18911,2025-02-27 23:00:00,5.7,4.8,94.0,0.0,NaN,0.0,0.0,9.3,1018.4,NaN,7.0,23


### Dni wolne od pracy

In [38]:
from datetime import datetime, timedelta
import pandas as pd
import holidays

# Get the date range from df_combined_1
start_date = df_combined_1['Data'].min()
end_date = df_combined_1['Data'].max()

# Create a date range
date_range = pd.date_range(start=start_date, end=end_date, freq='D')

# Create a dictionary of Polish holidays
pl_holidays = holidays.PL()

# Create a DataFrame with dates and holiday information
holiday_df = pd.DataFrame({
    'Data': date_range,
    'Dzień wolny': [1 if date in pl_holidays else 0 for date in date_range]
})

# Add weekend information (Saturday=6, Sunday=7)
holiday_df['Weekend'] = holiday_df['Data'].dt.dayofweek.apply(lambda x: 1 if x >= 5 else 0)

# Combine holiday and weekend information
holiday_df['Dzień wolny'] = holiday_df[['Dzień wolny', 'Weekend']].max(axis=1)

# Drop the weekend column as we don't need it anymore
holiday_df = holiday_df.drop('Weekend', axis=1)

# Display the first few rows
display(holiday_df)


,Data,Dzień wolny
0,2023-01-02,0
1,2023-01-03,0
2,2023-01-04,0
3,2023-01-05,0
4,2023-01-06,1
...,...,...
784,2025-02-24,0
785,2025-02-25,0
786,2025-02-26,0
787,2025-02-27,0


### Wschód i zachód słońca

In [39]:
start_date = df_combined_1['Data'].min()
end_date = df_combined_1['Data'].max()

# Create a date range
date_range = pd.date_range(start=start_date, end=end_date, freq='D')
date_range

DatetimeIndex(['2023-01-02', '2023-01-03', '2023-01-04', '2023-01-05',
               '2023-01-06', '2023-01-07', '2023-01-08', '2023-01-09',
               '2023-01-10', '2023-01-11',
               ...
               '2025-02-19', '2025-02-20', '2025-02-21', '2025-02-22',
               '2025-02-23', '2025-02-24', '2025-02-25', '2025-02-26',
               '2025-02-27', '2025-02-28'],
              dtype='datetime64[ns]', length=789, freq='D')

In [40]:
# Wybór lokalizacji (np. Warszawa)
city = LocationInfo("Warszawa", "Polska", "Europe/Warsaw", 52.2298, 21.0122)
# Pobranie danych o słońcu dla dzisiejszego dnia
# Initialize empty lists for sunrise and sunset times
sunrise_times = []
sunset_times = []

# Calculate sun data for each date in date_range
for date in date_range:
    s = sun(city.observer, date=date.date())
    sunrise_times.append(s['sunrise'] + timedelta(hours=1))
    sunset_times.append(s['sunset'] + timedelta(hours=1))

# Create a DataFrame with the calculated times
# Create initial DataFrame with dates and sun times
sun_df = pd.DataFrame({
    'Data': date_range,
    'Wschód słońca': [t.time() for t in sunrise_times],
    'Zachód słońca': [t.time() for t in sunset_times]
})

# Create a DataFrame with hourly intervals
hourly_df = pd.DataFrame()
for date in date_range:
    for hour in range(24):
        hourly_df = pd.concat([hourly_df, pd.DataFrame({
            'Data': [date],
            'Godzina': [hour],
            'Data_godzina': [date.replace(hour=hour)]
        })])

# Reset index
hourly_df = hourly_df.reset_index(drop=True)

# Merge with sun_df
hourly_df = pd.merge(hourly_df, sun_df, on='Data')

# Convert times to comparable format
hourly_df['sun'] = hourly_df.apply(
    lambda row: 1 if (row['Godzina'] >= row['Wschód słońca'].hour and 
                     row['Godzina'] < row['Zachód słońca'].hour) else 0,
    axis=1
)
hourly_df["Date_only"] = hourly_df["Data"].dt.date
hourly_df

,Data,Godzina,Data_godzina,Wschód słońca,Zachód słońca,sun,Date_only
0,2023-01-02,0,2023-01-02 00:00:00,07:45:31.433247,15:34:21.705523,0,2023-01-02
1,2023-01-02,1,2023-01-02 01:00:00,07:45:31.433247,15:34:21.705523,0,2023-01-02
2,2023-01-02,2,2023-01-02 02:00:00,07:45:31.433247,15:34:21.705523,0,2023-01-02
3,2023-01-02,3,2023-01-02 03:00:00,07:45:31.433247,15:34:21.705523,0,2023-01-02
4,2023-01-02,4,2023-01-02 04:00:00,07:45:31.433247,15:34:21.705523,0,2023-01-02
...,...,...,...,...,...,...,...
18931,2025-02-28,19,2025-02-28 19:00:00,06:24:16.386723,17:13:28.468997,0,2025-02-28
18932,2025-02-28,20,2025-02-28 20:00:00,06:24:16.386723,17:13:28.468997,0,2025-02-28
18933,2025-02-28,21,2025-02-28 21:00:00,06:24:16.386723,17:13:28.468997,0,2025-02-28
18934,2025-02-28,22,2025-02-28 22:00:00,06:24:16.386723,17:13:28.468997,0,2025-02-28


In [41]:
df_temperature.columns

Index(['Data', 'Temperatura', 'dwpt', 'rhum', 'prcp', 'snow', 'wdir', 'wspd',
       'wpgt', 'pres', 'tsun', 'coco', 'Godzina'],
      dtype='object')

### Łaczenie tabel

In [42]:
df_combined_1.columns

Index(['Maksymalna temperatura dobowa [°C]',
       'Minimalna temperatura dobowa [°C]', 'Średnia temperatura dobowa [°C]',
       'Temperatura minimalna przy gruncie [°C]', 'Suma dobowa opadu [mm]',
       'Rodzaj opadu [S/W/ ]', 'Wysokość pokrywy śnieżnej [cm]',
       'Równoważnik wodny śniegu  [mm/cm]', 'Usłonecznienie [godziny]',
       'Czas trwania opadu deszczu [godziny]',
       'Czas trwania opadu śniegu [godziny]',
       'Czas trwania opadu deszczu ze śniegiem [godziny]',
       'Czas trwania gradu [godziny]', 'Czas trwania mgły [godziny]',
       'Czas trwania zamglenia  [godziny]', 'Czas trwania sadzi [godziny]',
       'Czas trwania gołoledzi [godziny]',
       'Czas trwania zamieci śnieżnej niskiej [godziny]',
       'Czas trwania zamieci śnieżnej wysokiej [godziny]',
       'Czas trwania zmętnienia [godziny]',
       'Czas trwania wiatru >=10m/s [godziny]',
       'Czas trwania wiatru >15m/s [godziny]', 'Czas trwania burzy  [godziny]',
       'Czas trwania rosy  [godzi

In [46]:
# Read the merged dataframe
merged_df = pd.read_csv("merged_df.csv")
merged_df['Data'] = pd.to_datetime(merged_df['Data'])
merged_df['Godzina'] = merged_df['Data'].dt.hour
merged_df['Date_only'] = merged_df['Data'].dt.date
merged_df['Dzień tygodnia'] = merged_df['Data'].dt.day_name()



df_combined_1['Date_only'] = pd.to_datetime(df_combined_1['Data']).dt.date
df_temperature['Date_only'] = df_temperature['Data'].dt.date

# Merge the temperature dataframe
merged_df = pd.merge(merged_df, df_combined_1, 
                     on=['Date_only', "Godzina"], how='left', suffixes=('_merged', '_combined'))

# Merge with temperature from library
merged_df = pd.merge(merged_df, df_temperature,
                     on=['Date_only', "Godzina"], how='left', suffixes=('', '_temp'))

# Convert Date_only in holiday_df to date type for merging
holiday_df['Date_only'] = holiday_df['Data'].dt.date

# Merge with holiday information
merged_df = pd.merge(merged_df, holiday_df[['Date_only', 'Dzień wolny']], 
                     on=['Date_only'], how='left')

# Merge with day information
merged_df = pd.merge(merged_df, hourly_df[['Date_only', 'sun', "Godzina"]], 
                     on=['Date_only', "Godzina"], how='left')

# Drop the temporary date column
merged_df.drop('Date_only', axis=1, inplace=True)

# Save the final merged dataframe
merged_df.to_csv('merged_df.csv', index=False)


In [ ]:
# # Merge the temperature dataframe
# merged_df = pd.merge(merged_df, df_combined_1, 
#                      on=['Date_only', "Godzina"], how='left', suffixes=('_merged', '_combined'))

# # Merge with temperature from library
# merged_df = pd.merge(merged_df, df_temperature,
#                      on=['Date_only', "Godzina"], how='left', suffixes=('', '_temp'))

# # Convert Date_only in holiday_df to date type for merging
# holiday_df['Date_only'] = holiday_df['Data'].dt.date

# # Merge with holiday information
# merged_df = pd.merge(merged_df, holiday_df[['Date_only', 'Dzień wolny']], 
#                      on=['Date_only'], how='left')

# # Merge with day information
# merged_df = pd.merge(merged_df, hourly_df[['Date_only', 'sun', "Godzina"]], 
#                      on=['Date_only', "Godzina"], how='left')

# # Drop the temporary date column
# merged_df.drop('Date_only', axis=1, inplace=True)

# # Save the final merged dataframe
# merged_df.to_csv('merged_df.csv', index=False)


,Data_x,value,forecast,Godzina,Maksymalna temperatura dobowa [°C],Minimalna temperatura dobowa [°C],Średnia temperatura dobowa [°C],Temperatura minimalna przy gruncie [°C],Suma dobowa opadu [mm],Rodzaj opadu [S/W/ ],...,prcp,snow,wdir,wspd,wpgt,pres,tsun,coco,Dzień wolny,sun
0,2024-06-19 00:00:00,15716.9615,15600.0,0,17.5,13.4,15.5,13.3,0.6,W,...,0.0,NaN,130.0,7.2,18.5,1012.8,NaN,4.0,0,0
1,2024-06-19 01:00:00,15073.0545,15200.0,1,17.5,13.4,15.5,13.3,0.6,W,...,0.0,NaN,150.0,7.2,18.5,1012.2,NaN,2.0,0,0
2,2024-06-19 02:00:00,14819.3095,15150.0,2,17.5,13.4,15.5,13.3,0.6,W,...,0.0,NaN,200.0,7.2,18.5,1011.9,NaN,2.0,0,0
3,2024-06-19 03:00:00,14804.3095,15025.0,3,17.5,13.4,15.5,13.3,0.6,W,...,0.0,NaN,210.0,10.8,18.5,1013.5,NaN,2.0,0,1
4,2024-06-19 04:00:00,14947.8790,14650.0,4,17.5,13.4,15.5,13.3,0.6,W,...,0.0,NaN,300.0,14.4,16.7,1013.6,NaN,2.0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13243,2024-06-17 19:00:00,17250.0000,NaN,19,21.9,16.0,17.7,15.1,7.6,W,...,0.0,NaN,150.0,7.2,14.8,1013.7,NaN,2.0,0,0
13244,2024-06-17 20:00:00,16950.0000,NaN,20,21.9,16.0,17.7,15.1,7.6,W,...,0.0,NaN,160.0,3.6,14.8,1014.3,NaN,2.0,0,0
13245,2024-06-17 21:00:00,16600.0000,NaN,21,21.9,16.0,17.7,15.1,7.6,W,...,0.0,NaN,150.0,7.2,14.8,1014.3,NaN,2.0,0,0
13246,2024-06-17 22:00:00,16250.0000,NaN,22,21.9,16.0,17.7,15.1,7.6,W,...,0.0,NaN,230.0,3.6,13.0,1014.8,NaN,17.0,0,0


In [ ]:
import seaborn as sns

import matplotlib.pyplot as plt

# Calculate correlation coefficient
correlation = merged_df['Średnia temperatura dobowa [°C]'].corr(merged_df['value'])

# Create scatter plot
plt.figure(figsize=(10, 6))
sns.scatterplot(data=merged_df, x='Średnia temperatura dobowa [°C]', y='value', alpha=0.5)
plt.title(f'Correlation between Average Temperature and Value\nCorrelation coefficient: {correlation:.3f}')
plt.grid(True)
plt.show()

NameError: name 'merged_df' is not defined

In [ ]:
columns_to_drop

Index(['Czas trwania opadu deszczu [godziny]',
       'Czas trwania opadu śniegu [godziny]',
       'Czas trwania opadu deszczu ze śniegiem [godziny]',
       'Czas trwania gradu [godziny]', 'Czas trwania mgły [godziny]',
       'Czas trwania zamglenia  [godziny]', 'Czas trwania sadzi [godziny]',
       'Czas trwania gołoledzi [godziny]',
       'Czas trwania zamieci śnieżnej niskiej [godziny]',
       'Czas trwania zamieci śnieżnej wysokiej [godziny]',
       'Czas trwania zmętnienia [godziny]',
       'Czas trwania wiatru >=10m/s [godziny]',
       'Czas trwania wiatru >15m/s [godziny]', 'Czas trwania burzy  [godziny]',
       'Czas trwania rosy  [godziny]', 'Czas trwania szronu [godziny]'],
      dtype='object')

In [ ]:
# Drop rows with NaN values in numeric columns
clean_df = merged_df.select_dtypes(include=['float64', 'int64']).dropna()

# Calculate correlation matrix
correlation_matrix = clean_df.corr()

# Create a heatmap
plt.figure(figsize=(20, 16))
mask = np.triu(np.ones_like(correlation_matrix), k=1)  # Mask upper triangle
# Remove unwanted columns from axis labels
# Filter out columns containing '[godziny]'
# Filter out columns containing '[godziny]' and duplicate temperature columns
columns_to_drop = correlation_matrix.columns[
    correlation_matrix.columns.str.contains('Czas trwania', na=False)
]
correlation_matrix_filtered = correlation_matrix.drop(columns_to_drop, axis=0).drop(columns_to_drop, axis=1)

# Create new mask with correct shape
mask = np.triu(np.ones_like(correlation_matrix_filtered), k=1)

sns.heatmap(correlation_matrix_filtered, annot=True, cmap='coolwarm', center=0, fmt='.2f', 
            annot_kws={'size': 8}, mask=mask)
plt.title('Correlation Matrix', fontsize=14)
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()




NameError: name 'plt' is not defined

In [ ]:
# For categorical columns (chi-square test)
categorical_columns = merged_df.select_dtypes(include=['object']).columns
print("\nCorrelation analysis between categorical variables and 'value':")

for col in categorical_columns:
    contingency = pd.crosstab(merged_df[col], pd.qcut(merged_df['value'], q=4))
    chi2, p_value = chi2_contingency(contingency)[:2]
    print(f"\n{col} vs value:")
    print(f"Chi-square statistic: {chi2:.2f}")
    print(f"p-value: {p_value:.4f}")



Correlation analysis between categorical variables and 'value':

Rodzaj opadu [S/W/ ] vs value:
Chi-square statistic: 222.55
p-value: 0.0000

Stan gruntu [Z/R] vs value:
Chi-square statistic: 137.23
p-value: 0.0000
